In [2]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [3]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [4]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [5]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[3]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_V_only
target lang: English verb (WN) [en_V_only]
target_class: 


In [6]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq         = 2
abuse_threshold      = 0.05 # larger value selects shorter units, smaller value selects longer units
min_bot_size         = 3
# number of terms listed for a given topic
n_terms_to_show = 60

In [7]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[0]
ngram_is_inclusive  = True
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary       = False
boundary_mark      = "#"
## term setting
gap_mark           = "…"
term_is_skippy     = True
n_for_ngram        = 4
max_gap_ratio      = 1.0
max_gap_size       = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents   = True
accent_marks       = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: spell
term_is_skippy: True
max_gap_size: 11
n_for_ngram: 4
accent_status: 
term_type: sp_skippy4gram


In [8]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [9]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [10]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_V_only.csv


,spell,sound
3758,shell,ˈʃɛɫ
2397,level,ˈɫɛvəɫ
4388,towel,ˈtaʊəɫ
511,bulletin,ˈbʊɫɪtən
919,convey,kənˈveɪ
2932,perpetuate,pɝˈpɛtʃəˌweɪt
541,butter,ˈbətɝ
2009,hectograph,ˈhɛktəˌɡɹæf
3139,promise,ˈpɹɑməs
1865,gladden,ˈɡɫædən


In [11]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
           spell      sound
3281    reappear     ɹiəpɪɹ
424       borrow      bɑɹoʊ
905      content    kɑntɛnt
890      consort    kənsɔɹt
1547       excel      ɪksɛɫ
...          ...        ...
227   backstroke  bækstɹoʊk
1135     delight     dɪɫaɪt
422       border      bɔɹdɝ
721        claim      kɫeɪm
1064        daub        dɔb

[2374 rows x 2 columns]


In [12]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
3281,reappear,ɹiəpɪɹ,"[r, e, a, p, p, e, a, r]",8,0,0,"[ɹ, i, ə, p, ɪ, ɹ]",6
424,borrow,bɑɹoʊ,"[b, o, r, r, o, w]",6,0,0,"[b, ɑ, ɹ, o, ʊ]",5
905,content,kɑntɛnt,"[c, o, n, t, e, n, t]",7,0,0,"[k, ɑ, n, t, ɛ, n, t]",7
890,consort,kənsɔɹt,"[c, o, n, s, o, r, t]",7,0,0,"[k, ə, n, s, ɔ, ɹ, t]",7
1547,excel,ɪksɛɫ,"[e, x, c, e, l]",5,0,0,"[ɪ, k, s, ɛ, ɫ]",5
...,...,...,...,...,...,...,...,...
227,backstroke,bækstɹoʊk,"[b, a, c, k, s, t, r, o, k, e]",10,0,0,"[b, æ, k, s, t, ɹ, o, ʊ, k]",9
1135,delight,dɪɫaɪt,"[d, e, l, i, g, h, t]",7,0,0,"[d, ɪ, ɫ, a, ɪ, t]",6
422,border,bɔɹdɝ,"[b, o, r, d, e, r]",6,0,0,"[b, ɔ, ɹ, d, ɝ]",5
721,claim,kɫeɪm,"[c, l, a, i, m]",5,0,0,"[k, ɫ, e, ɪ, m]",5


In [13]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sp_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
3281,reappear,ɹiəpɪɹ,"[r, e, a, p, p, e, a, r]",8,0,0,"[ɹ, i, ə, p, ɪ, ɹ]",6
424,borrow,bɑɹoʊ,"[b, o, r, r, o, w]",6,0,0,"[b, ɑ, ɹ, o, ʊ]",5
905,content,kɑntɛnt,"[c, o, n, t, e, n, t]",7,0,0,"[k, ɑ, n, t, ɛ, n, t]",7
890,consort,kənsɔɹt,"[c, o, n, s, o, r, t]",7,0,0,"[k, ə, n, s, ɔ, ɹ, t]",7
1547,excel,ɪksɛɫ,"[e, x, c, e, l]",5,0,0,"[ɪ, k, s, ɛ, ɫ]",5
...,...,...,...,...,...,...,...,...
227,backstroke,bækstɹoʊk,"[b, a, c, k, s, t, r, o, k, e]",10,0,0,"[b, æ, k, s, t, ɹ, o, ʊ, k]",9
1135,delight,dɪɫaɪt,"[d, e, l, i, g, h, t]",7,0,0,"[d, ɪ, ɫ, a, ɪ, t]",6
422,border,bɔɹdɝ,"[b, o, r, d, e, r]",6,0,0,"[b, ɔ, ɹ, d, ɝ]",5
721,claim,kɫeɪm,"[c, l, a, i, m]",5,0,0,"[k, ɫ, e, ɪ, m]",5


In [14]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

2340

In [15]:
## spell 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)

if term_class == 'spell':
    #sp_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sp_1gram'] ]
    sp_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_2grams):
            g.extend(list(df['sp_1gram'])[i])
    ## add sp_2gram
    df['sp_2gram'] = sp_2grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/3511996102.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_2gram'] = sp_2grams


In [16]:
## spell 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sp_1gram'] ]
    sp_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_3grams):
            g.extend(list(df['sp_2gram'])[i])
    ## add sp_3gram
    df['sp_3gram'] = sp_3grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/1405656247.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_3gram'] = sp_3grams


In [17]:
## spell 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sp_1gram'] ]
    sp_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_4grams):
            g.extend(list(df['sp_3gram'])[i])
    ## add sp_4gram
    df['sp_4gram'] = sp_4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/1571879696.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_4gram'] = sp_4grams


In [18]:
## spell 5grams
#import ngrams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_5grams):
            g.extend(list(df['sp_4gram'])[i])
    ## add sp_5gram
    df['sp_5gram'] = sp_5grams

In [19]:
## spell skippy 2gram
#import ngrams_skippy
import gen_ngrams
reload_module = False
module_name = "gen_ngrams"
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'spell':
    #sp_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy2grams):
            g.extend(list(df['sp_1gram'])[i])
    #
    df['sp_skippy2gram'] = sp_skippy2grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/2729548111.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy2gram'] = sp_skippy2grams


In [20]:
## spell skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy3grams):
            g.extend(list(df['sp_skippy2gram'])[i])
    #
    df['sp_skippy3gram'] = sp_skippy3grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/2630798607.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy3gram'] = sp_skippy3grams


In [21]:
## spell skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy4grams):
            g.extend(list(df['sp_skippy3gram'])[i])
    #
    df['sp_skippy4gram'] = sp_skippy4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/1288956926.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy4gram'] = sp_skippy4grams


In [22]:
## spell skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy5grams):
            g.extend(list(df['sp_skippy4gram'])[i])
    #
    df['sp_skippy5gram'] = sp_skippy5grams

In [23]:
## sound 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'sound':
    #sn_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sn_1gram'] ]
    sn_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep ="", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_2grams):
            g.extend(list(df['sn_1gram'])[i])
    ## add sn_2gram
    df['sn_2gram'] = sn_2grams

In [24]:
## sound 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sn_1gram'] ]
    sn_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_3grams):
            g.extend(list(df['sn_2gram'])[i])
    ## add sn_3gram
    df['sn_3gram'] = sn_3grams

In [25]:
## sound 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sn_1gram'] ]
    sn_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_4grams):
            g.extend(list(df['sn_3gram'])[i])
    ## add sn_4gram
    df['sn_4gram'] = sn_4grams

In [26]:
## sound 5grams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_5grams):
            g.extend(list(df['sn_4gram'])[i])
    ## add sn_4gram
    df['sn_5gram'] = sn_5grams

In [27]:
## sound skippy 2gram
#import ngrams_skippy
import gen_ngrams
if term_class == 'sound':
    #sn_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy2grams):
            g.extend(list(df['sn_1gram'])[i])
    #
    df['sn_skippy2gram'] = sn_skippy2grams

In [28]:
## sound skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy3grams):
            g.extend(list(df['sn_skippy2gram'])[i])
    #
    df['sn_skippy3gram'] = sn_skippy3grams

In [29]:
## sound skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy4grams):
            g.extend(list(df['sn_skippy3gram'])[i])
    #
    df['sn_skippy4gram'] = sn_skippy4grams

In [30]:
## sound skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy5grams):
            g.extend(list(df['sn_skippy4gram'])[i])
    #
    df['sn_skippy5gram'] = sn_skippy5grams

In [31]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram
3281,reappear,ɹiəpɪɹ,"[r, e, a, p, p, e, a, r]","[re, ea, ap, pp, pe, ea, ar, r, e, a, p, p, e,...","[rea, eap, app, ppe, pea, ear, re, ea, ap, pp,...","[reap, eapp, appe, ppea, pear, rea, eap, app, ...","[re, r…a, r…p, r…p, r…e, r…a, r…r, ea, e…p, e…...","[rea, re…p, re…p, re…e, re…a, re…r, r…ap, r…a…...","[reap, rea…p, rea…e, rea…a, rea…r, re…pp, re…p..."
424,borrow,bɑɹoʊ,"[b, o, r, r, o, w]","[bo, or, rr, ro, ow, b, o, r, r, o, w]","[bor, orr, rro, row, bo, or, rr, ro, ow, b, o,...","[borr, orro, rrow, bor, orr, rro, row, bo, or,...","[bo, b…r, b…r, b…o, b…w, or, o…r, o…o, o…w, rr...","[bor, bo…r, bo…o, bo…w, b…rr, b…r…o, b…r…w, b…...","[borr, bor…o, bor…w, bo…ro, bo…r…w, bo…ow, b…r..."
905,content,kɑntɛnt,"[c, o, n, t, e, n, t]","[co, on, nt, te, en, nt, c, o, n, t, e, n, t]","[con, ont, nte, ten, ent, co, on, nt, te, en, ...","[cont, onte, nten, tent, con, ont, nte, ten, e...","[co, c…n, c…t, c…e, c…n, c…t, on, o…t, o…e, o…...","[con, co…t, co…e, co…n, co…t, c…nt, c…n…e, c…n...","[cont, con…e, con…n, con…t, co…te, co…t…n, co…..."
890,consort,kənsɔɹt,"[c, o, n, s, o, r, t]","[co, on, ns, so, or, rt, c, o, n, s, o, r, t]","[con, ons, nso, sor, ort, co, on, ns, so, or, ...","[cons, onso, nsor, sort, con, ons, nso, sor, o...","[co, c…n, c…s, c…o, c…r, c…t, on, o…s, o…o, o…...","[con, co…s, co…o, co…r, co…t, c…ns, c…n…o, c…n...","[cons, con…o, con…r, con…t, co…so, co…s…r, co…..."
1547,excel,ɪksɛɫ,"[e, x, c, e, l]","[ex, xc, ce, el, e, x, c, e, l]","[exc, xce, cel, ex, xc, ce, el, e, x, c, e, l]","[exce, xcel, exc, xce, cel, ex, xc, ce, el, e,...","[ex, e…c, e…e, e…l, xc, x…e, x…l, ce, c…l, el,...","[exc, ex…e, ex…l, e…ce, e…c…l, e…el, xce, xc…l...","[exce, exc…l, ex…el, e…cel, xcel, xcel, exc, e..."
...,...,...,...,...,...,...,...,...,...
227,backstroke,bækstɹoʊk,"[b, a, c, k, s, t, r, o, k, e]","[ba, ac, ck, ks, st, tr, ro, ok, ke, b, a, c, ...","[bac, ack, cks, kst, str, tro, rok, oke, ba, a...","[back, acks, ckst, kstr, stro, trok, roke, bac...","[ba, b…c, b…k, b…s, b…t, b…r, b…o, b…k, b…e, a...","[bac, ba…k, ba…s, ba…t, ba…r, ba…o, ba…k, ba…e...","[back, bac…s, bac…t, bac…r, bac…o, bac…k, bac…..."
1135,delight,dɪɫaɪt,"[d, e, l, i, g, h, t]","[de, el, li, ig, gh, ht, d, e, l, i, g, h, t]","[del, eli, lig, igh, ght, de, el, li, ig, gh, ...","[deli, elig, ligh, ight, del, eli, lig, igh, g...","[de, d…l, d…i, d…g, d…h, d…t, el, e…i, e…g, e…...","[del, de…i, de…g, de…h, de…t, d…li, d…l…g, d…l...","[deli, del…g, del…h, del…t, de…ig, de…i…h, de…..."
422,border,bɔɹdɝ,"[b, o, r, d, e, r]","[bo, or, rd, de, er, b, o, r, d, e, r]","[bor, ord, rde, der, bo, or, rd, de, er, b, o,...","[bord, orde, rder, bor, ord, rde, der, bo, or,...","[bo, b…r, b…d, b…e, b…r, or, o…d, o…e, o…r, rd...","[bor, bo…d, bo…e, bo…r, b…rd, b…r…e, b…r…r, b…...","[bord, bor…e, bor…r, bo…de, bo…d…r, bo…er, b…r..."
721,claim,kɫeɪm,"[c, l, a, i, m]","[cl, la, ai, im, c, l, a, i, m]","[cla, lai, aim, cl, la, ai, im, c, l, a, i, m]","[clai, laim, cla, lai, aim, cl, la, ai, im, c,...","[cl, c…a, c…i, c…m, la, l…i, l…m, ai, a…m, im,...","[cla, cl…i, cl…m, c…ai, c…a…m, c…im, lai, la…m...","[clai, cla…m, cl…im, c…aim, laim, laim, cla, c..."


In [32]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_38880/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(1857, 'nibble'),
 (1877, 'school'),
 (2040, 'stylize'),
 (1215, 'remake'),
 (2061, 'sneeze'),
 (1475, 'reprint'),
 (647, 'greet'),
 (834, 'burp'),
 (809, 'brandish'),
 (274, 'badger')]

In [33]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

(changed) term_type: sp_skippy4gram


[['peer',
  'pee',
  'pe…r',
  'p…er',
  'eer',
  'eer',
  'pe',
  'p…e',
  'p…r',
  'ee',
  'e…r',
  'er',
  'ee',
  'e…r',
  'er',
  'er',
  'p',
  'e',
  'e',
  'r']]

In [34]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<80470 unique tokens: ['a', 'ap', 'app', 'appe', 'app…a']...>
term filtering applied
Dictionary<28194 unique tokens: ['ap', 'app', 'appe', 'app…a', 'app…r']...>


In [35]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

/Users/kowk/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/kowk/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/kowk/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/kowk/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/kowk/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires ver

In [36]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [37]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

documents_topics = np.zeros([hdp15.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp15[c]:
        documents_topics[topic_id][doc_id] = prob

n_docs_to_show = 10
n_terms_to_show = 60
hdp15.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp15.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.001 * ac + 0.001 * ck + 0.001 * r…te + 0.001 * r…a…e + 0.001 * i…at + 0.001 * i…ate + 0.001 * a…k + 0.001 * n…te + 0.001 * na + 0.001 * or + 0.001 * na…e + 0.001 * o…a…e + 0.001 * ack + 0.001 * n…a + 0.001 * r…at + 0.001 * t…a + 0.001 * o…at + 0.001 * m…t + 0.001 * r…ate + 0.001 * ba + 0.001 * d…a + 0.001 * e…te + 0.001 * ra…e + 0.001 * e…o + 0.001 * nat + 0.001 * o…te + 0.001 * r…n…e + 0.001 * nate + 0.001 * ce + 0.001 * e…ate + 0.0 * o…ate + 0.0 * t…n + 0.0 * d…t + 0.0 * c…te + 0.0 * m…ate + 0.0 * e…at + 0.0 * g…t + 0.0 * an + 0.0 * ta + 0.0 * h…e + 0.0 * m…a + 0.0 * r…g + 0.0 * u…a + 0.0 * on + 0.0 * in…e + 0.0 * ag + 0.0 * b…t + 0.0 * s…a…e + 0.0 * a…a + 0.0 * m…a…e + 0.0 * o…n + 0.0 * u…c + 0.0 * ca + 0.0 * m…n + 0.0 * g…te + 0.0 * i…n + 0.0 * t…te + 0.0 * es + 0.0 * s…n + 0.0 * o…s
nonzero count:  871
	0.9993: recriminate
	0.9993: incriminate
	0.9993: predominate
	0.9993: premeditate
	0.9993: concentrate
	0.9993: contaminate
	0.9992: communicate
	0.9992: necessitate

In [38]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp15.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp15_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp15_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics-max_ntop{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp15_topics_out, header = False, index = None)

In [39]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [40]:
## save LDAvis output as a html file
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
save_LDAvis = True
if save_LDAvis:
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [41]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp45.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp45_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp45_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics-max_ntop{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp45_topics_out, header = False, index = None)

In [42]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp45.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp45[c]:
        documents_topics[topic_id][doc_id] = prob
#
n_docs_to_show = 10
n_terms_to_show = 7
hdp45.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp45.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * l…te + 0.002 * i…ate + 0.002 * i…at + 0.002 * m…r + 0.002 * ce + 0.001 * ur + 0.001 * d…r
nonzero count: 618
	0.9993: reintroduce
	0.9993: premeditate
	0.9992: contaminate
	0.9992: overproduce
	0.9992: commiserate
	0.9992: orchestrate
	0.9992: proliferate
	0.9992: hallucinate
	0.9991: superimpose
	0.9991: regurgitate
topic_id 1: 0.002 * nt + 0.002 * ze + 0.002 * is + 0.001 * iz + 0.001 * ize + 0.001 * i…i + 0.001 * i…i…e
nonzero count: 393
	0.9992: disorganize
	0.9992: romanticize
	0.9992: misconstrue
	0.9991: necessitate
	0.9991: investigate
	0.9991: discontinue
	0.9990: guesstimate
	0.9988: dehumanize
	0.9987: pressurize
	0.9986: discipline
topic_id 2: 0.001 * i…ate + 0.001 * i…at + 0.001 * r…te + 0.001 * nd + 0.001 * r…a…e + 0.001 * p…a + 0.001 * an
nonzero count: 302
	0.9992: recriminate
	0.9992: incriminate
	0.9992: precipitate
	0.9992: appropriate
	0.9991: certificate
	0.9988: understand
	0.9988: appreciate
	0.9987: decapitate
	0.9987: inseminate
	0.9987: reap

In [43]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, T = max_n_topics,
                               random_state = 1,
                               #var_converge = 0.001
                               )
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [ ]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [ ]:
## save topic structures
hdp_topics = hdp90.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp90_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp90_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics-max_ntop{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp90_topics_out, header = False, index = None)

In [ ]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp90.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp90[c]:
        documents_topics[topic_id][doc_id] = prob
## investigate topics
n_docs_to_show  = 10
n_terms_to_show = 7
hdp90.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = hdp90.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * z# + 0.002 * ɪz + 0.002 * ɪz# + 0.002 * a…z + 0.002 * a…z# + 0.002 * aɪz# + 0.002 * aɪz
nonzero count:  220
	0.9993: #stɛɹəɫaɪz#
	0.9992: #mɪnɝəɫaɪz#
	0.9992: #spɛʃəɫaɪz#
	0.9992: #ɪmpɹəvaɪz#
	0.9992: #fɛdɝəɫaɪz#
	0.9992: #tɹænsɫeɪt#
	0.9991: #æmpjəteɪt#
	0.9991: #tæbjəɫeɪt#
	0.9991: #kwɑntɪfaɪ#
	0.9991: #pɝsɑnəfaɪ#
topic_id 1: 0.003 * z# + 0.002 * kt + 0.002 * ɪz + 0.002 * kt# + 0.002 * ɪz# + 0.002 * aɪz# + 0.002 * a…z
nonzero count:  133
	0.9992: #mæksəmaɪz#
	0.9992: #ɹændəmaɪz#
	0.9992: #kəstəmaɪz#
	0.9992: #dɪskənɛkt#
	0.9991: #kənstɹɪkt#
	0.9991: #pɹɑsəkjut#
	0.9991: #hɑɹmənaɪz#
	0.9991: #kaʊntɝækt#
	0.9989: #kɑntɹækt#
	0.9988: #mɛkənaɪz#
topic_id 2: 0.001 * kə + 0.001 * kə…# + 0.001 * ɪ# + 0.001 * əm…# + 0.001 * əm + 0.001 * bə + 0.001 * bə…#
nonzero count:  102
	0.9991: #kɑnskɹɪpt#
	0.9990: #sɪŋkəpeɪt#
	0.9990: #ɪŋkəɫkeɪt#
	0.9988: #dɪɡnəfaɪ#
	0.9988: #sɪɡnəfaɪ#
	0.9987: #mækɹəmeɪ#
	0.9987: #ɹɛkəmɛnd#
	0.9987: #ɪkspɫeɪn#
	0.9986: #ɔtəmoʊbiɫ#
	0.9985: #kənfɹən